# Reconocimiento de Entidades Nombradas
## Ejercicio Práctico - Procesamiento de Lenguaje Natural

**Objetivos de Aprendizaje:**
1. Implementar NER usando modelos pre-entrenados en español
2. Crear interfaces interactivas con Gradio
3. Comparar enfoques: Transformers vs API Gemini
4. Desarrollar prototipos rápidos para aplicaciones de PLN

---
**Entorno recomendado:** Google Colab o Amazon SageMaker Studio

**Tiempo estimado:** 60-90 minutos

## Instalación de Dependencias

In [ ]:
# Instalación de librerías necesarias
%%capture
!pip install -q transformers torch gradio google-genai

In [ ]:
# Verificar instalación
import sys
print(f"Python: {sys.version}")
print("Todas las dependencias instaladas correctamente")

## Configuración de APIs

### Para Google Colab:
1. Andá a la barra lateral izquierda y hacé clic en 🔑 (Secrets)
2. Agrega una nueva clave: `GOOGLE_API_KEY`
3. Pega tu API key de Google AI Studio

### Para SageMaker Studio:
1. Configura las variables de entorno en tu instancia
2. O usa el método de input manual más abajo

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración de API Key para Gemini
try:
    # Método 1: Google Colab Secrets
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("API Key cargada desde Google Colab Secrets")
except:
    # Método 2: Variables de entorno (SageMaker)
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY:
        print("API Key cargada desde variables de entorno")
    else:
        print("No se encontró GOOGLE_API_KEY")
        print("Podes continuar solo con la parte de Transformers")

---
# PARTE 1: NER con Transformers de Hugging Face

Utilizaremos un modelo especializado en español que puede identificar:
- **PER**: Personas
- **LOC**: Lugares
- **ORG**: Organizaciones  
- **MISC**: Misceláneo

In [ ]:
from transformers import pipeline
import torch

# Verificar disponibilidad de GPU
device = 0 if torch.cuda.is_available() else -1
print(f"🖥️  Dispositivo: {'GPU' if device == 0 else 'CPU'}")

# Cargar modelo de NER en español
print("📥 Cargando modelo de NER para español...")
MODEL_NAME = "mrm8488/bert-spanish-cased-finetuned-ner"

try:
    ner_pipeline = pipeline(
        "ner",
        model=MODEL_NAME,
        aggregation_strategy="simple",
        device=device
    )
    print(f"Modelo {MODEL_NAME} cargado exitosamente")
except Exception as e:
    print(f"❌ Error al cargar modelo: {e}")
    ner_pipeline = None

In [ ]:
# Texto de ejemplo con contexto argentino
texto_ejemplo = """
Hola, soy María González y trabajo en la Universidad de Buenos Aires.
Vivo en el barrio de San Telmo y mi empresa favorita es MercadoLibre.
La semana pasada visité el Obelisco con mi amigo Carlos Pérez,
quien trabaja en Google Argentina. Nos encontramos en la estación
Constitución del subte y fuimos a comer un asado en La Boca.
"""

In [ ]:
def analizar_entidades_transformers(texto):
    """Procesa texto y extrae entidades usando Transformers"""
    if not ner_pipeline:
        return []

    entidades = ner_pipeline(texto)

    # Formatear resultados
    resultados = []
    for ent in entidades:
        resultados.append({
            'texto': ent['word'],
            'etiqueta': ent['entity_group'],
            'confianza': round(ent['score'], 3),
            'posicion': (ent['start'], ent['end'])
        })

    return resultados

In [ ]:
# Probar el modelo
print("🔍 Analizando texto de ejemplo...")
print(f"📝 Texto: {texto_ejemplo.strip()}")
print("\n📊 Entidades encontradas:")

entidades_encontradas = analizar_entidades_transformers(texto_ejemplo)
for ent in entidades_encontradas:
    print(f"  • {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})")

---
# PARTE 2: NER con API de Gemini

Utilizaremos la API de Gemini para un análisis más detallado y contextual.

In [ ]:
# Configurar cliente Gemini
cliente_gemini = None

if GOOGLE_API_KEY:
    try:
        from google import genai
        cliente_gemini = genai.Client(api_key=GOOGLE_API_KEY)
        print("Cliente Gemini configurado correctamente")
    except Exception as e:
        print(f"Error al configurar Gemini: {e}")
else:
    print("API Key de Gemini no disponible")
    print("Podes obtener una gratis en: https://ai.google.dev/")

In [ ]:
def analizar_entidades_gemini(texto):
    """Analiza entidades usando Gemini API"""
    if not cliente_gemini:
        return "❌ Cliente Gemini no disponible"

    prompt = f"""
    Extraé todas las entidades nombradas del siguiente texto en español argentino y clasificálas:

    CATEGORÍAS:
    - PERSONA: Nombres de personas
    - LUGAR: Ciudades, países, barrios, direcciones, lugares específicos
    - ORGANIZACIÓN: Empresas, universidades, instituciones
    - MISCELÁNEO: Otros nombres propios (productos, eventos, marcas)

    FORMATO DE RESPUESTA:
    [ENTIDAD] → [CATEGORÍA] → [BREVE EXPLICACIÓN]

    TEXTO A ANALIZAR:
    {texto}
    """

    try:
        respuesta = cliente_gemini.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt]
        )
        return respuesta.text
    except Exception as e:
        return f"❌ Error: {e}"

# Probar Gemini si está disponible
if cliente_gemini:
    print("🔍 Analizando con Gemini...")
    resultado_gemini = analizar_entidades_gemini(texto_ejemplo)
    print("\nAnálisis de Gemini:")
    print(resultado_gemini)
else:
    print("⏭️  Saltando análisis con Gemini (API Key no disponible)")

---
# PARTE 3: Interfaces Interactivas con Gradio

Crearemos interfaces web interactivas para probar nuestros modelos.

In [ ]:
import gradio as gr

def interfaz_ner_transformers(texto):
    """Interfaz para el modelo de Transformers"""
    if not texto.strip():
        return {"text": "Ingresa un texto para analizar", "entities": []}

    if not ner_pipeline:
        return {"text": "Modelo no disponible", "entities": []}

    # Procesar con Transformers
    entidades = ner_pipeline(texto)

    # Formatear para Gradio HighlightedText
    entidades_gradio = []
    for ent in entidades:
        entidades_gradio.append({
            "entity": ent["entity_group"],
            "word": ent["word"],
            "start": ent["start"],
            "end": ent["end"],
            "score": ent["score"]
        })

    return {"text": texto, "entities": entidades_gradio}

# Ejemplos para la interfaz
ejemplos_arg = [
    "Me llamo Juan Pérez y trabajo en el Banco Nación en Buenos Aires.",
    "Cristina Kirchner fue presidenta de Argentina y vive en Santa Cruz.",
    "River Plate jugará contra Boca Juniors en el estadio Monumental.",
    "Lionel Messi nació en Rosario y jugó en el Barcelona.",
    "La Universidad de La Plata es muy prestigiosa en Argentina."
]

# Crear interfaz
demo_transformers = gr.Interface(
    fn=interfaz_ner_transformers,
    inputs=[
        gr.Textbox(
            label="📝 Texto a analizar",
            placeholder="Escribe aquí tu texto en español...",
            lines=4
        )
    ],
    outputs=[
        gr.HighlightedText(
            label="🎯 Entidades Identificadas",
            show_legend=True
        )
    ],
    title="NER con Transformers - Español (de argentina)",
    description="""
    **Modelo:** `mrm8488/bert-spanish-cased-finetuned-ner`

    Identifica entidades nombradas en textos en español:
    - 🧑 **PER**: Personas
    - 🌍 **LOC**: Lugares
    - 🏢 **ORG**: Organizaciones
    - 📦 **MISC**: Misceláneo
    """,
    examples=ejemplos_arg,
    allow_flagging="never",
    theme=gr.themes.Soft()
)

print("✅ Interfaz de Transformers creada")

In [ ]:
# Interfaz para Gemini (solo si está disponible)
if cliente_gemini:
    def interfaz_ner_gemini(texto):
        """Interfaz para Gemini API"""
        if not texto.strip():
            return "Ingresa un texto para analizar"
        return analizar_entidades_gemini(texto)

    demo_gemini = gr.Interface(
        fn=interfaz_ner_gemini,
        inputs=[
            gr.Textbox(
                label="📝 Texto a analizar",
                placeholder="Escribe aquí tu texto en español...",
                lines=4
            )
        ],
        outputs=[
            gr.Textbox(
                label="🧠 Análisis de Gemini",
                lines=10
            )
        ],
        title="NER con Gemini - Análisis Detallado",
        description="""
        **Modelo:** Google Gemini 2.0 Flash

        Análisis avanzado de entidades nombradas con explicaciones contextuales
        optimizado para español argentino.
        """,
        examples=ejemplos_arg,
        allow_flagging="never",
        theme=gr.themes.Soft()
    )
    print("✅ Interfaz de Gemini creada")
else:
    print("⏭️  Interfaz de Gemini no creada (API Key no disponible)")

In [ ]:
# Interfaz comparativa (solo si ambos están disponibles)
if ner_pipeline and cliente_gemini:
    def comparar_modelos(texto):
        """Compara resultados de ambos modelos"""
        if not texto.strip():
            return "Ingresa texto para comparar", "Ingresa texto para comparar"

        # Resultado Transformers
        entidades_tf = analizar_entidades_transformers(texto)
        resultado_tf = "TRANSFORMERS:\n\n"
        for ent in entidades_tf:
            resultado_tf += f"• {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})\n"

        # Resultado Gemini
        resultado_gemini = "GEMINI:\n\n" + analizar_entidades_gemini(texto)

        return resultado_tf, resultado_gemini

    demo_comparativo = gr.Interface(
        fn=comparar_modelos,
        inputs=[
            gr.Textbox(
                label="📝 Texto a comparar",
                placeholder="Ingresa texto para ver la comparación...",
                lines=3
            )
        ],
        outputs=[
            gr.Textbox(label="Transformers", lines=8),
            gr.Textbox(label="Gemini", lines=8)
        ],
        title="⚔️ Comparación: Transformers vs Gemini",
        description="Compara los resultados de ambos enfoques lado a lado.",
        examples=[
            "Diego Maradona jugó en Boca Juniors y en el Napoli de Italia.",
            "El gobierno argentino anunció medidas desde Casa Rosada."
        ],
        allow_flagging="never"
    )
    print("✅ Interfaz comparativa creada")
else:
    print("⏭️  Interfaz comparativa no creada (requiere ambos modelos)")

## 🚀 Lanzar Interfaces

Ejecuta las celdas siguientes para lanzar las interfaces interactivas:

In [ ]:
# Lanzar interfaz de Transformers
if ner_pipeline:
    print("🚀 Lanzando interfaz de Transformers...")
    demo_transformers.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: modelo de Transformers no disponible")

In [ ]:
# Lanzar interfaz de Gemini
if cliente_gemini:
    print("🚀 Lanzando interfaz de Gemini...")
    demo_gemini.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: API de Gemini no disponible")

In [ ]:
# Lanzar interfaz comparativa
if ner_pipeline and cliente_gemini:
    print("🚀 Lanzando interfaz comparativa...")
    demo_comparativo.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: requiere ambos modelos disponibles")

---
# 🎓 EJERCICIOS

## 📝 Ejercicio 1: Personalización (BÁSICO)
1. Modifica los ejemplos para incluir más contexto argentino específico
2. Agrega 3 ejemplos nuevos con nombres de barrios porteños
3. Probá con texto de diferentes regiones de Argentina

## 🔧 Ejercicio 2: Análisis Comparativo (INTERMEDIO)
1. Crea una función que cuente cuántas entidades encuentra cada modelo
2. Implementa un sistema de métricas de tiempo de procesamiento
3. Analiza en qué casos cada modelo funciona mejor

## 🚀 Ejercicio 3: Extensiones Avanzadas (AVANZADO)
1. Implementa procesamiento en lote de múltiples textos
2. Crea una función de exportación de resultados a CSV
3. Desarrolla un sistema de filtrado por tipo de entidad

## 💡 Proyecto Integrador
Elegí una de estas aplicaciones y desarróllala:
- **Analizador de noticias argentinas**: Extrae personas y lugares de artículos
- **Procesador de CVs**: Identifica nombres, empresas y universidades
- **Análisis de redes sociales**: Detecta menciones de políticos y lugares

## 🤔 Preguntas de Reflexión
1. ¿Cuáles son las ventajas y desventajas de cada enfoque?
2. ¿En qué casos usarías un modelo local vs una API?
3. ¿Cómo evaluarías la precisión de los resultados?
4. ¿Qué consideraciones éticas debemos tener en cuenta?
5. ¿Cómo escalarías esta solución para procesar miles de documentos?

In [ ]:
# 📝 ESPACIO PARA TUS EJERCICIOS
# Usa esta celda para experimentar y desarrollar tus soluciones

# Ejemplo: Función para contar entidades por tipo
def contar_entidades_por_tipo(texto):
    """Cuenta entidades por categoría usando Transformers"""
    if not ner_pipeline:
        return {}

    entidades = ner_pipeline(texto)
    conteo = {}

    for ent in entidades:
        tipo = ent['entity_group']
        if tipo in conteo:
            conteo[tipo] += 1
        else:
            conteo[tipo] = 1

    return conteo

# Probar la función
texto_prueba = "Juan Pérez trabaja en Google Argentina en Buenos Aires con María López."
print("📊 Conteo de entidades:")
print(contar_entidades_por_tipo(texto_prueba))

# TODO: Agrega aquí tus propias funciones y experimentos

---
# 🎯 Conclusión

¡Felicitaciones! Completaste el ejercicio de Reconocimiento de Entidades Nombradas.

## 📚 Lo que aprendiste:
- ✅ Implementar NER con modelos pre-entrenados
- ✅ Usar APIs de IA generativa para tareas de PLN
- ✅ Crear interfaces interactivas con Gradio
- ✅ Comparar diferentes enfoques de NER

## 🔄 Próximos pasos:
1. Experimentá con otros modelos de Hugging Face
2. Probá con textos de diferentes dominios
3. Implementa tu proyecto integrador
4. Compartí tus resultados con la clase

## 📖 Recursos adicionales:
- [Hugging Face Models](https://huggingface.co/models?pipeline_tag=token-classification&language=es)
- [Gradio Documentation](https://gradio.app/docs/)
- [Google AI Studio](https://ai.google.dev/)

---
**¡Éxito en tu trabajo integrador!** 🎓🚀